- 使用Gradio + colab打造一個簡單的網頁


In [3]:
!pip install gradio

- 這是Gradio的map程式範例

In [ ]:
import gradio as gr
import plotly.graph_objects as go
from datasets import load_dataset

dataset = load_dataset("gradio/NYC-Airbnb-Open-Data", split="train")
df = dataset.to_pandas()

def filter_map(min_price, max_price, boroughs):
    """
    Filters the Airbnb data based on price range and boroughs and creates a Plotly map.

    Args:
        min_price (float): The minimum price for filtering.
        max_price (float): The maximum price for filtering.
        boroughs (list): A list of selected boroughs.

    Returns:
        plotly.graph_objects.Figure: A Plotly figure object representing the map.
    """
    filtered_df = df[(df['neighbourhood_group'].isin(boroughs)) &
          (df['price'] > min_price) & (df['price'] < max_price)]
    names = filtered_df["name"].tolist()
    prices = filtered_df["price"].tolist()
    text_list = [(names[i], prices[i]) for i in range(0, len(names))]
    fig = go.Figure(go.Scattermapbox(
            customdata=text_list,
            lat=filtered_df['latitude'].tolist(),
            lon=filtered_df['longitude'].tolist(),
            mode='markers',
            marker=go.scattermapbox.Marker(
                size=6
            ),
            hoverinfo="text",
            hovertemplate='<b>Name</b>: %{customdata[0]}<br><b>Price</b>: $%{customdata[1]}'
        ))

    fig.update_layout(
        mapbox_style="open-street-map",
        hovermode='closest',
        mapbox=dict(
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=40.67,
                lon=-73.90
            ),
            pitch=0,
            zoom=9
        ),
    )

    return fig

with gr.Blocks() as demo:
    with gr.Column():
        with gr.Row():
            min_price = gr.Number(value=250, label="Minimum Price")
            max_price = gr.Number(value=1000, label="Maximum Price")
        boroughs = gr.CheckboxGroup(choices=["Queens", "Brooklyn", "Manhattan", "Bronx", "Staten Island"], value=["Queens", "Brooklyn"], label="Select Boroughs:")
        btn = gr.Button(value="Update Filter")
        map = gr.Plot()
    demo.load(filter_map, [min_price, max_price, boroughs], map)
    btn.click(filter_map, [min_price, max_price, boroughs], map)

if __name__ == "__main__":
    demo.launch()

- 我們利用gemini AI改寫成我們需要的AQI資訊的地圖

In [4]:
import gradio as gr
import plotly.graph_objects as go
import requests
import pandas as pd

# 定義資料來源 URL 和 API Key
url = "https://data.moenv.gov.tw/api/v2/aqx_p_432?api_key=9e565f9a-84dd-4e79-9097-d403cae1ea75&limit=1000&sort=ImportDate%20desc&format=JSON"

# 定義區域與測站的對應關係 (這是一個假設的對應，您可能需要根據實際情況調整)
# 這個對應關係可以手動建立，或者從資料中提取區域資訊（如果有的話）
# 這裡我們先假設一些測站屬於特定區域
region_mapping = {
    "北部": ["基隆", "汐止", "萬里", "新店", "土城", "板橋", "新莊", "菜寮", "林口", "淡水", "士林", "大同", "中山", "古亭", "松山", "萬華", "永和", "竹北", "新竹", "頭份"],
    "中部": ["苗栗", "豐原", "沙鹿", "大里", "忠明", "西屯", "彰化", "線西", "斗六", "崙背", "竹山"],
    "南部": ["嘉義", "臺南", "安南", "善化", "新營", "橋頭", "岡山", "左營", "楠梓", "仁武", "前鎮", "復興", "小港", "大寮", "潮州", "屏東", "恆春"],
    "東部": ["臺東", "花蓮", "宜蘭", "冬山"]
}

# 載入資料並進行前處理
def load_pm25_data():
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()['records']
        df = pd.DataFrame(data)

        # 確保包含必要的欄位並轉換資料類型
        required_columns = ['sitename', 'county', 'latitude', 'longitude', 'pm2.5']
        if not all(col in df.columns for col in required_columns):
            print("資料中缺少必要的測站名稱、縣市、經緯度或 pm2.5 欄位。")
            return pd.DataFrame() # 回傳空的 DataFrame

        df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
        df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
        df['pm2.5'] = pd.to_numeric(df['pm2.5'], errors='coerce')

        # 移除經緯度或 pm2.5 為 NaN 的列
        df_cleaned = df.dropna(subset=['latitude', 'longitude', 'pm2.5'])

        # 添加區域資訊 (根據 sitename 對應到區域)
        df_cleaned['region'] = df_cleaned['sitename'].apply(
            lambda x: next((region for region, sites in region_mapping.items() if x in sites), "其他區域")
        )

        return df_cleaned
    except Exception as e:
        print(f"載入資料時發生錯誤：{e}")
        return pd.DataFrame() # 回傳空的 DataFrame

# 在應用程式啟動時先載入資料
df_pm25 = load_pm25_data()

# 過濾資料並生成地圖的函式
def filter_pm25_map(min_pm25, max_pm25, regions):
    """
    根據 PM2.5 數值範圍和區域過濾資料，並創建 Plotly 地圖。

    Args:
        min_pm25 (float): 最小的 PM2.5 數值。
        max_pm25 (float): 最大的 PM2.5 數值。
        regions (list): 選擇的區域列表。

    Returns:
        plotly.graph_objects.Figure: 表示地圖的 Plotly 圖形物件。
        str: 如果資料為空，則回傳錯誤訊息。
    """
    if df_pm25.empty:
        return go.Figure(), "無法載入 PM2.5 資料，請稍後再試。"

    # 根據選擇的區域過濾測站
    selected_sites = []
    for region in regions:
        selected_sites.extend(region_mapping.get(region, []))

    # 過濾資料框
    filtered_df = df_pm25[
        (df_pm25['sitename'].isin(selected_sites)) &
        (df_pm25['pm2.5'] >= min_pm25) &
        (df_pm25['pm2.5'] <= max_pm25)
    ]

    if filtered_df.empty:
        return go.Figure(), "沒有符合篩選條件的資料點可以顯示。"

    # 準備地圖上的 hover 資訊
    site_names = filtered_df["sitename"].tolist()
    pm25_values = filtered_df["pm2.5"].tolist()
    text_list = [(site_names[i], pm25_values[i]) for i in range(len(site_names))]

    # 計算地圖中心點 (使用過濾後的資料點的平均經緯度)
    center_lat = filtered_df['latitude'].mean()
    center_lon = filtered_df['longitude'].mean()

    fig = go.Figure(go.Scattermapbox(
            customdata=text_list,
            lat=filtered_df['latitude'].tolist(),
            lon=filtered_df['longitude'].tolist(),
            mode='markers',
            marker=go.scattermapbox.Marker(
                size=8,
                color=filtered_df['pm2.5'], # 使用 PM2.5 值作為顏色
                colorscale="Viridis", # 設定顏色比例尺
                showscale=True,
                colorbar=dict(title="PM2.5")
            ),
            hoverinfo="text",
            hovertemplate='<b>測站</b>: %{customdata[0]}<br><b>PM2.5</b>: %{customdata[1]}'
        ))

    fig.update_layout(
        mapbox_style="open-street-map",
        hovermode='closest',
        mapbox=dict(
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=center_lat,
                lon=center_lon
            ),
            pitch=0,
            zoom=7 # 調整縮放級別以適應台灣地圖
        ),
        margin={"r":0,"t":0,"l":0,"b":0} # 調整地圖邊距
    )

    return fig, "" # 回傳地圖物件和空的錯誤訊息

# 建立 Gradio 介面
with gr.Blocks() as demo:
    gr.Markdown("## 台灣 PM2.5 測站資料視覺化")
    with gr.Column():
        with gr.Row():
            min_pm25 = gr.Number(value=0, label="最小 PM2.5 數值")
            max_pm25 = gr.Number(value=100, label="最大 PM2.5 數值")
        regions = gr.CheckboxGroup(choices=["北部", "中部", "南部", "東部"], value=["北部", "中部", "南部", "東部"], label="選擇區域:")
        btn = gr.Button(value="更新地圖")
        map_output = gr.Plot()
        error_output = gr.Textbox(label="訊息", interactive=False)

    # 設定事件觸發
    demo.load(filter_pm25_map, [min_pm25, max_pm25, regions], [map_output, error_output])
    btn.click(filter_pm25_map, [min_pm25, max_pm25, regions], [map_output, error_output])

# 啟動 Gradio 應用程式
if __name__ == "__main__":
    demo.launch()

<ipython-input-4-acea7ba6e9c2>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['region'] = df_cleaned['sitename'].apply(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3e4f354962e4fcf045.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 接著再利用AI把時間資訊也整合到程式碼中

In [5]:
import gradio as gr
import plotly.graph_objects as go
import requests
import pandas as pd

# 定義資料來源 URL 和 API Key
url = "https://data.moenv.gov.tw/api/v2/aqx_p_432?api_key=9e565f9a-84dd-4e79-9097-d403cae1ea75&limit=1000&sort=ImportDate%20desc&format=JSON"

# 定義區域與測站的對應關係 (這是一個假設的對應，您可能需要根據實際情況調整)
region_mapping = {
    "北部": ["基隆", "汐止", "萬里", "新店", "土城", "板橋", "新莊", "菜寮", "林口", "淡水", "士林", "大同", "中山", "古亭", "松山", "萬華", "永和", "竹北", "新竹", "頭份"],
    "中部": ["苗栗", "豐原", "沙鹿", "大里", "忠明", "西屯", "彰化", "線西", "斗六", "崙背", "竹山"],
    "南部": ["嘉義", "臺南", "安南", "善化", "新營", "橋頭", "岡山", "左營", "楠梓", "仁武", "前鎮", "復興", "小港", "大寮", "潮州", "屏東", "恆春"],
    "東部": ["臺東", "花蓮", "宜蘭", "冬山"]
}

# 載入資料並進行前處理
def load_pm25_data():
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()['records']
        df = pd.DataFrame(data)

        # 確保包含必要的欄位並轉換資料類型
        required_columns = ['sitename', 'county', 'latitude', 'longitude', 'pm2.5', 'publishtime']
        if not all(col in df.columns for col in required_columns):
            print("資料中缺少必要的測站名稱、縣市、經緯度、pm2.5 或 publishtime 欄位。")
            return pd.DataFrame() # 回傳空的 DataFrame

        df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
        df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
        df['pm2.5'] = pd.to_numeric(df['pm2.5'], errors='coerce')

        # 移除經緯度、pm2.5 或 publishtime 為 NaN 的列
        df_cleaned = df.dropna(subset=['latitude', 'longitude', 'pm2.5', 'publishtime'])

        # 添加區域資訊 (根據 sitename 對應到區域)
        df_cleaned['region'] = df_cleaned['sitename'].apply(
            lambda x: next((region for region, sites in region_mapping.items() if x in sites), "其他區域")
        )

        return df_cleaned
    except Exception as e:
        print(f"載入資料時發生錯誤：{e}")
        return pd.DataFrame() # 回傳空的 DataFrame

# 在應用程式啟動時先載入資料
df_pm25 = load_pm25_data()

# 過濾資料並生成地圖的函式
def filter_pm25_map(min_pm25, max_pm25, regions):
    """
    根據 PM2.5 數值範圍和區域過濾資料，並創建 Plotly 地圖。

    Args:
        min_pm25 (float): 最小的 PM2.5 數值。
        max_pm25 (float): 最大的 PM2.5 數值。
        regions (list): 選擇的區域列表。

    Returns:
        plotly.graph_objects.Figure: 表示地圖的 Plotly 圖形物件。
        str: 如果資料為空，則回傳錯誤訊息。
    """
    if df_pm25.empty:
        return go.Figure(), "無法載入 PM2.5 資料，請稍後再試。"

    # 根據選擇的區域過濾測站
    selected_sites = []
    for region in regions:
        selected_sites.extend(region_mapping.get(region, []))

    # 過濾資料框
    filtered_df = df_pm25[
        (df_pm25['sitename'].isin(selected_sites)) &
        (df_pm25['pm2.5'] >= min_pm25) &
        (df_pm25['pm2.5'] <= max_pm25)
    ].copy() # 使用 .copy() 避免 SettingWithCopyWarning

    if filtered_df.empty:
        return go.Figure(), "沒有符合篩選條件的資料點可以顯示。"

    # 準備地圖上的 hover 資訊
    site_names = filtered_df["sitename"].tolist()
    pm25_values = filtered_df["pm2.5"].tolist()
    publish_times = filtered_df["publishtime"].tolist() # 獲取 publishtime 列表
    text_list = [(site_names[i], pm25_values[i], publish_times[i]) for i in range(len(site_names))] # 將 publishtime 加入到 customdata 中

    # 計算地圖中心點 (使用過濾後的資料點的平均經緯度)
    center_lat = filtered_df['latitude'].mean()
    center_lon = filtered_df['longitude'].mean()

    fig = go.Figure(go.Scattermapbox(
            customdata=text_list,
            lat=filtered_df['latitude'].tolist(),
            lon=filtered_df['longitude'].tolist(),
            mode='markers',
            marker=go.scattermapbox.Marker(
                size=8,
                color=filtered_df['pm2.5'], # 使用 PM2.5 值作為顏色
                colorscale="Viridis", # 設定顏色比例尺
                showscale=True,
                colorbar=dict(title="PM2.5")
            ),
            hoverinfo="text",
            hovertemplate='<b>測站</b>: %{customdata[0]}<br><b>PM2.5</b>: %{customdata[1]}<br><b>發布時間</b>: %{customdata[2]}' # 修改 hover 模板以顯示 publishtime
        ))

    fig.update_layout(
        mapbox_style="open-street-map",
        hovermode='closest',
        mapbox=dict(
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=center_lat,
                lon=center_lon
            ),
            pitch=0,
            zoom=7
        ),
        margin={"r":0,"t":0,"l":0,"b":0}
    )

    # 獲取資料的最新發布時間 (假設資料已按時間降序排列)
    latest_publish_time = df_pm25['publishtime'].iloc[0] if not df_pm25.empty else "無法獲取"
    message = f"資料發布時間：{latest_publish_time}"

    return fig, message # 回傳地圖物件和訊息

# 建立 Gradio 介面
with gr.Blocks() as demo:
    gr.Markdown("## 台灣 PM2.5 測站資料視覺化")
    with gr.Column():
        with gr.Row():
            min_pm25 = gr.Number(value=0, label="最小 PM2.5 數值")
            max_pm25 = gr.Number(value=100, label="最大 PM2.5 數值")
        regions = gr.CheckboxGroup(choices=["北部", "中部", "南部", "東部"], value=["北部", "中部", "南部", "東部"], label="選擇區域:")
        btn = gr.Button(value="更新地圖")
        map_output = gr.Plot()
        error_output = gr.Textbox(label="訊息", interactive=False) # 訊息欄位

    # 設定事件觸發
    demo.load(filter_pm25_map, [min_pm25, max_pm25, regions], [map_output, error_output])
    btn.click(filter_pm25_map, [min_pm25, max_pm25, regions], [map_output, error_output])

# 啟動 Gradio 應用程式
if __name__ == "__main__":
    demo.launch()

<ipython-input-5-866ff92de4ed>:39: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://36fc4ba6962ec57184.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
